In [ ]:
%pylab inline
%autosave 20
import os
os.environ['KERAS_BACKEND']='theano'
import keras
import sys
sys.path.append('/home/peter')
sys.path.append('/home/peter/code/projects/MultiNEAT')
sys.path.append('/home/ubuntu')
from universal import *
sys.path.append('/home/peter/code/projects')
sys.path.append('/home/peter/code/projects/deepneat')
sys.path.append('/home/ubuntu')
sys.path.append('/home/ubuntu/new/automl')
from aidevutil import *
from keras.utils.vis_utils import plot_model
from tqdm import tqdm_notebook as tqdm
from deepneat import *

In [ ]:
from dask import compute, delayed, persist
from dask.distributed import Client, wait
from dask.distributed import as_completed

In [ ]:
import ipywidgets as widgets

In [ ]:
(x_train, y_train), (x_test, y_test) = get_mnist(size=(28,28))
dx = x_train.reshape(x_train.shape[0], -1)
dy = y_train

dx_test = x_test.reshape(x_test.shape[0], -1)

In [ ]:
rf = np.arange(dx.shape[1])
rf = rnd.sample(rf.tolist(), 16)
dx = dx[:, rf]
dx_test = dx_test[:, rf]

In [ ]:
###################
# Prototype Nodes #
###################
#
# Linear, PCA
#
# LogisticRegression
#
#

In [ ]:
# NEAT Parameters 
params = NEAT.Parameters()

params.PopulationSize = 300
params.DynamicCompatibility = True
params.YoungAgeTreshold = 3
params.SpeciesMaxStagnation = 10
params.OldAgeTreshold = 10
params.MinSpecies = 1
params.MaxSpecies = 12
params.RouletteWheelSelection = False
params.ArchiveEnforcement = False
params.InnovationsForever = True

params.ConstraintTrials = 128

params.MutateAddNeuronProb = 0.15/3
params.MutateAddLinkProb = 0.5/3
params.MutateRemLinkProb = 0.1/3
params.RecurrentProb = 0.0
params.MaxWeight = 1.0

params.MutateWeightsProb = 0.0
params.MutateActivationAProb = 0.0
params.MutateActivationBProb = 0.0
params.MutateNeuronTimeConstantsProb = 0.0
params.MutateNeuronBiasesProb = 0.0

params.MutateGenomeTraitsProb = 0.0
params.MutateNeuronTraitsProb = 0.5
params.MutateLinkTraitsProb = 0.0

params.OverallMutationRate = 0.5
params.CrossoverRate = 0.7
params.MultipointCrossoverRate = 0.4
params.SurvivalRate = 0.2
params.InterspeciesCrossoverRate = 0.005
params.PreferFitterParentRate = 0.5

params.DontUseBiasNeuron = True
params.AllowLoops = False
params.AllowClones = True

params.ExcessCoeff = 1.0
params.DisjointCoeff = 1.0

params.WeightDiffCoeff = 0.0
params.TimeConstantDiffCoeff = 0.0
params.BiasDiffCoeff = 0.0
params.ActivationADiffCoeff = 0.0
params.ActivationBDiffCoeff = 0.0
params.NormalizeGenomeSize = True

params.MinCompatTreshold = 0.0
params.CompatTreshold = 1.25
params.CompatTreshChangeInterval_Evaluations = 1
params.CompatTresholdModifier = 0.002

In [ ]:
# This is the custom trait
class BaseObject:
    def __init__(self):
        pass
        
    def mate(self, other):
        baby = BaseObject()
        return baby
    
    def mutate(self):
        baby = BaseObject()
        return baby
    
    def distance_to(self, other):
        return rnd.uniform(0,1)

In [ ]:
class genePCA(BaseObject):
    def __init__(self):
        print('Derived1 init called.')
    
    def mate(self, other):
        print('Derived1 mate called')
        return Derived1()
    
    def mutate(self):
        print('Derived1 mutate called')
        return Derived1()
    
    def distance_to(self, other):
        print('Derived1 distance_to called')
        return rnd.uniform(0,1)

In [ ]:
class geneLinear(BaseObject):
    def __init__(self):
        self.x = rnd.uniform(0,1)
    
    def mate(self, other):
        return Derived2()
    
    def mutate(self):
        return Derived2()
    
    def distance_to(self, other):
        return rnd.uniform(0,1)

In [ ]:
class geneLogisticRegression(BaseObject):
    def __init__(self):
        self.x = rnd.uniform(0,1)
    
    def mate(self, other):
        return Derived2()
    
    def mutate(self):
        return Derived2()
    
    def distance_to(self, other):
        return rnd.uniform(0,1)

In [ ]:
derived_list = [Derived1, Derived2]
probs = [1.0]*len(derived_list)

In [ ]:
# Set traits

# merge mode
s = ['concat',
    # when dimensions match, these can be done, otherwise it defaults to concat
     'add','mul','avg','min','max',
    ]
p = [1.0] * len(s)
mm = {'details': {'set': s, 'probs': p},
      'importance_coeff': 0.0,
      'mutation_prob': 0.3,
      'type': 'str'}

# node type
s = ['linear', 'pca']
p = [1.0] * len(s)
nt = {'details': {'set': s, 'probs': p},
      'importance_coeff': 0.2,
      'mutation_prob': 0.4,
      'type': 'str'}

params.SetNeuronTraitParameters('mm', mm)
params.SetNeuronTraitParameters('nt', nt)

In [ ]:
node = {'details': (derived_list, probs),
          'importance_coeff': 0.1,
          'mutation_prob': 0.2,
          'type': 'pyclassset'}

params.SetNeuronTraitParameters('node', node)

In [ ]:
num_inputs = 1
num_outputs = 1

In [ ]:
num_input_dims = dx.shape[1]
num_output_dims = 1

In [ ]:
g = NEAT.Genome(0, num_inputs, 1, num_outputs, 0, NEAT.ActivationFunction.RELU,
                    NEAT.ActivationFunction.RELU, 0, params, 0, 1)

In [ ]:
pop = NEAT.Population(g, params, True, 1.0, 0)
pop.RNG.Seed(int(time.clock()*100))

In [ ]:
pop.Species[0].Individuals[0].GetNeuronTraits()[-1][-1]['node'].x

In [ ]:
pop.Parameters.AllowClones = False

In [ ]:
gr = NEAT.Genome2NX(g)

In [ ]:
def activate_graph(gr, inputs, num_outputs=1):
    """
    allnodes = list(nx.dfs_postorder_nodes(gr))[::-1]
    for a in allnodes: gr.node[a]['act'] = None
    
    # separate input from non-input nodes
    #onlyinputs = sorted([x for x in allnodes if x <= num_inputs], reverse=True)
    allnodes = [x for x in allnodes if x > num_inputs]
    
    # input the data
    for i,inp in zip(range(1, num_inputs+1), inputs): 
        gr.node[i]['act'] = inp.reshape(-1)
        
    # pass through the graph
    for an in allnodes:
        #print(gr.node[a], end=' ')
        mm = gr.node[an]['mm']
        
        #slope = gr.node[an]['slope']
        
        # collect the inputs to this node
        
        # also sort the incoming edges by id for consistency
        inedg = list(gr.in_edges(an))
        #inedg = sorted(inedg, key = lambda x: x[0])
        inps = [gr.node[i]['act'] for i,o in inedg]
        inps = np.vstack(inps)
        
        sact = np.sum(inps, axis=0) # this node's default activation

        if 1:
            if mm=='add':
                act = sact
            if mm=='neg':
                act = -sact
            if mm=='mul':
                act = np.prod(inps, axis=0)
            if mm=='inv':
                act = 1.0 / sact
            if mm=='sqr':
                act = sact**2
            if mm=='cube':
                act = sact**3
            if mm=='sqrt':
                act = np.sqrt(sact)
            if mm=='sin':
                act = np.sin(sact)
            if mm=='cos':
                act = np.cos(sact)
            if mm=='log':
                act = np.log(sact)
            if mm=='exp':
                act = np.exp(sact)
            if mm=='abs':
                act = np.abs(sact)
            if mm=='tan':
                act = np.tan(sact)
            if mm=='min':
                act = np.min(inps, axis=0)
            if mm=='max':
                act = np.max(inps, axis=0)
            if mm=='mean':
                act = np.mean(inps, axis=0)
            if mm=='argmax':
                act = np.argmax(inps, axis=0)
            if mm=='smax':
                s = np.exp(inps) / np.sum(np.exp(inps))
                act = s[np.argmax(s, axis=0)]
            if mm=='var':
                act = np.var(inps, axis=0)
            if mm=='std':
                act = np.std(inps, axis=0)
        
        #except:
        #    act = sact

        # apply slope and bias
        act = act + bias
        #act = act.reshape(1,-1)
        
        # store activation
        gr.node[an]['act'] = act 
        
    #print('act:', act)
    outputs = [gr.node[o]['act'] for o in allnodes[-num_outputs:]]
    """
    return np.array([0.0])#outputs

In [ ]:
ak = dx.T
ak.shape

In [ ]:
imshow(NEAT.viz.Draw(g));

In [ ]:
#g.GetNeuronTraits()

In [ ]:
#gr.node[3]['mm']='sqr'

In [ ]:
# evaluation function

def evaluate(args):
    idx, gr, dx, dy = args
    datata = np.vstack([dx.T] + [np.ones((dx.shape[0],), dtype=np.float64)*x 
                                 for x in consts])
    at = activate_graph(gr, datata)[0]
    #at = at[~np.isnan(at)]# = 0
    #at = at[~np.isinf(at)]# = 0
    f = 100000000.0 - np.mean(np.abs(at.reshape(-1) - dy.reshape(-1)))
    f = np.clip(f, 0.0, 100000000.0)            
    return idx, f

def decide(x):
    return 100000000.0-x

In [ ]:
if debugxy:
    # now test a simple evolutionary run
    def target(x, y, *args):
        return x**2 - y#**2

    args = [(float(i-50),#rnd.uniform(-100,100), 
             float(i-50),#rnd.uniform(-100,100),
            ) + tuple(consts) for i in range(100)]
    gt = np.array([target(*(x)) for x in args])
    inn = np.array(args).T

    def evaluate(aaargs):
        idx, grr, dx, dy = aaargs

        at = activate_graph(grr, inn)[0]
        #at[np.isnan(at)] = 0
        #at[np.isinf(at)] = 0
        err = 100000000.0 - np.mean(np.abs(gt.reshape(-1) - at.reshape(-1)))
        err = np.clip(err, 0.0, 100000000.0) 
        #print(err)

        return idx, err
    
    def decide(x):
        return 100000000.0-x

In [ ]:
if debugxy:
    print(gt.shape, inn.shape)

In [ ]:
#gt

In [ ]:
#at

In [ ]:
if socket.gethostname() == 'desktop':
    cluster='192.168.0.108:8786'
else:
    cluster='172.31.43.76:8786'

In [ ]:
import time
st = time.time()
print('Connecting..')
if cluster:
    client = Client(cluster)
else:
    client = Client()
# push the data to the cluster
print('Scattering data to cluster..')
if not debugxy:
    fx = client.scatter(dx, broadcast=True, direct=True)
    fy = client.scatter(dy, broadcast=True, direct=True)
else:
    fx, fy = None, None
print(f'done in {time.time()-st} seconds')

In [ ]:
#evaluate((0, NEAT.Genome2NX(pop.Species[0].Individuals[2]), dx, dy))

In [ ]:
display_whole_pop = 0
display_max_species = 8
initeval = 0
evaluations = 10000
display_pop_each = 80000
once = False
x_shape, ys = dx.shape[1], 1
population = params.PopulationSize

penalize_stangation = 1
penalize_stagnation_evals = 2400

evhist = []
best_ever = 0
best_gs = []

In [ ]:
def poplen(pop):
    return sum([len(x.Individuals) for x in pop.Species])

In [ ]:
def prettydict(d, nonl=False):
    i,t,di = d
    if not nonl:
        ks = '%d - %s: ' % (i, t)
    else:
        return '%d - %s' % (i, t)
    s = []
    for k,v in sorted(list(di.items())):
        if isinstance(v, float):
            s.append( '%s %3.3f' % (k[0:1], v) )
        else:
            s.append( '%s %s' % (k[0:1], v) )
    return ks + ', '.join(s)

In [ ]:
def species_display(pop):
    genomes = [x.GetLeader() for x in pop.Species][0:display_max_species]
    f, axes = plt.subplots(1, len(genomes), figsize=(len(genomes) * 4.5, 14))
    print('Species Representatives:')
    if len(genomes)>1:
        for i,(ax, g) in enumerate(zip(axes, genomes)):
            try:
                img = NEAT.viz.Draw(g, size=(300, 400))[:, 0:230]
                ax.imshow(img)
                ax.set_title('%3.6f | %3.2f%%' % (decide(g.GetFitness()),
                                                  (len(pop.Species[i].Individuals)/poplen(pop))*100 ))
                s = '\n'.join([prettydict(x) for x in g.GetNeuronTraits() if x[1] != 'input'][0:8])
                s += '\n=============\n'
                s += '\n'.join([prettydict(x, nonl=True) for x in g.GetLinkTraits()][0:8])
                ax.set_xlabel(s)
            except Exception as ex:
                print(ex)
    else:
        try:
            g = pop.GetBestGenome()
            img = NEAT.viz.Draw(g, size=(300, 400))[:, 0:230]
            axes.imshow(img)
            axes.set_title('%3.6f' % (decide(g.GetFitness())))
            s = '\n'.join([prettydict(x) for x in g.GetNeuronTraits() if x[1] != 'input'][0:8])
            s += '\n=============\n'
            s += '\n'.join([prettydict(x, nonl=True) for x in g.GetLinkTraits()][0:8])
            axes.set_xlabel(s)
        except Exception as ex:
            print(ex)
    f.tight_layout()
    plt.show()

In [ ]:
# rtNEAT code
try:
    if initeval:
        print('============================================================')
        print("Please wait for the initial evaluation to complete.")
        now = time.time()
        genome_list = []
        for s in pop.Species:
            for i in s.Individuals:
                genome_list.append(i)
        sys.stdout.flush()
        # turn them into NX networks
        grlist = [NEAT.Genome2NX(x) for x in genome_list]
        idlist = [x.GetID() for x in genome_list]
        args = list(zip(idlist,
                        grlist,
                        [fx]*len(grlist),
                        [fy]*len(grlist),
                        ))

        # evaluate all individuals
        fitnesses = [0] * len(grlist)
        vrs = [None] * len(grlist)
        bar = ProgressBar(initial_value=0, max_value=len(grlist))
        cp = [client.submit(evaluate, x) for x in args]
        fff = {}
        for i,ftr in enumerate(as_completed(cp)):
            idx, fitness = ftr.result()
            #fitnesses[idx] = fitness
            fff[idx] = fitness
            bar.update(i)
        bar.finish()
        
        for gnm in genome_list:
            gnm.SetFitness( fff[gnm.GetID()] )
            gnm.SetEvaluated()

        #NEAT.ZipFitness(genome_list, fitnesses)
        print("Evaluation took", '%3.2f' % (time.time() - now), "seconds.")
    else:
        # No initial evaluation, start off with small random fitness
        for s in pop.Species:
            for i in s.Individuals:
                i.SetFitness(np.random.rand()/1000)
        grlist = [NEAT.Genome2NX(x) for x in NEAT.GetGenomeList(pop)]
        idlist = [x.GetID() for x in NEAT.GetGenomeList(pop)]
        args = list(zip(idlist,
                        grlist,
                        [fx]*len(grlist),
                        [fy]*len(grlist),
                        ))
        cp = [client.submit(evaluate, x) for x in args]

    # get best fitness in population and print it
    glist = NEAT.GetGenomeList(pop)
    fitness_list = [x.GetFitness() for x in glist]
    bidx = argmax(fitness_list)
    best = fitness_list[bidx]
    if initeval: evhist.append(decide(best))
    if best > best_ever:
        print('NEW RECORD!')
        print('Evaluations:', 0,
              'Fitness:', decide(best),
              'Species:', len(pop.Species))
        best_gs.append(NEAT.Genome2NX(glist[bidx]))
        print('-------- nodes -------- ')
        print('\n'.join([str(x) for x in glist[bidx].GetNeuronTraits() if x[1] != 'input']))
        print('-------- links -------- ')
        print('\n'.join([str(x) for x in glist[bidx].GetLinkTraits()]))
        print('----------------------- ')
        best_ever = best
        if viz:
            species_display(pop)

    print('============================================================')
    print('rtNEAT phase')
    print('============================================================')

    format_custom_text = FormatCustomText('CTresh: %(ctr).3f Species: %(sp)d Last fitness: %(fitness).3f Variance (min/max/mean/std): %(vr)s',
        dict(
            sp=0,
            fitness=0.0,
            vr=str([]),
            ctr=0.0,
        ),)

    widgets = ['Evaluated: ', Counter('%(value)d'), ' ', format_custom_text,
               ' (', Timer(), ') ', RotatingMarker(), FileTransferSpeed(unit='individuals') ]
    bar = ProgressBar(widgets=widgets, initial_value=0, max_value=evaluations)


    # parallel continious loop
    seq = as_completed(cp)
    i=0
    olds = []
    under_evaluation_now = len(cp)
    for f in seq:
        if i >= evaluations:
            [x.cancel() for x in cp]
            break
        if i > 0: bar.update(i)
        i += 1

        # get result from evaluation
        idx, fitness = f.result()
        try:
            if idx not in olds:
                # set that individual's fitness
                pop.AccessGenomeByID(idx).SetFitness(fitness)
                pop.AccessGenomeByID(idx).SetEvaluated()
                under_evaluation_now -= 1

                thegenome = pop.AccessGenomeByID(idx)

                # get best fitness in population and print it
                evhist.append(decide(fitness))
                if fitness > best_ever:
                    print('NEW RECORD!')
                    print('Evaluations:', i,
                          'Fitness:', decide(fitness))
                    best_gs.append(NEAT.Genome2NX(thegenome))
                    print('-------- nodes -------- ')
                    print('\n'.join([str(x) for x in thegenome.GetNeuronTraits() if x[1] != 'input']))
                    print('-------- links -------- ')
                    print('\n'.join([str(x) for x in thegenome.GetLinkTraits()]))
                    print('----------------------- ')
                    best_ever = fitness
                    if viz:
                        species_display(pop)
                elif (i%display_pop_each)==0: 
                    if viz:
                        species_display(pop)
                    

            else:
                print(f'Fitness of genome #{idx} was not set.')
                
        except Exception as ex:
            print(ex)
        format_custom_text.update_mapping(fitness=decide(fitness),
                                          sp=len(pop.Species),
                                          ctr=pop.Parameters.CompatTreshold)
        
                
        # apply exploration pressure to species
        if penalize_stangation:
            maxfit = max([x.GetFitness() for x in NEAT.GetGenomeList(pop)])
            for s in pop.Species:
                if s.EvalsNoImprovement > penalize_stagnation_evals:
                    bf = max([x.GetFitness() for x in s.Individuals])
                    # make an exception if that species contains the best genome so far
                    if bf < maxfit:
                        for ind in s.Individuals:
                            ind.SetFitness(0.000000001)

        # create new baby and add to list of tasks
        # only add new babies if the population under evaluation is below N%
        if under_evaluation_now <= (population*(1/3)):
            if not once:
                once=True
                print(f'rtNEAT reproduction cycle started at evaluation #{i}')

            old = NEAT.Genome()
            baby = pop.Tick(old)
            
            for l in baby.LinkGenes: l.Weight = 0.1 # for prettier display    
            
            olds.append(old.GetID())
            newf = client.submit(evaluate, (baby.GetID(),
                                            NEAT.Genome2NX(baby),
                                            fx,
                                            fy,
                                            ))
            seq.add(newf)
            under_evaluation_now += 1
            
except KeyboardInterrupt:
    pass
    
except Exception as ex:
    print(ex)

client.restart(timeout=600)

In [ ]:
#client.restart(timeout=600)